In [1]:
!pip install --quiet jupyterlab-vim jupytex
!jupyter labextension enable

%load_ext autoreload
%autoreload 2

In [2]:
!pip install --quiet neo4j

In [3]:
import neo4j
print(neo4j.__version__)

5.22.0


In [4]:
import hneo4j

from hneo4j import to_str

# Force reload.
# import importlib
# importlib.reload(hneo4j)

# Neo4j

In [6]:
from neo4j import GraphDatabase, RoutingControl

URI = "neo4j://neo4j:7687"
#URI = "bolt://neo4j:7687"
AUTH = ("neo4j", "testtest")

# def add_friend(driver, name, friend_name):
#     driver.execute_query(
#         "MERGE (a:Person {name: $name}) "
#         "MERGE (friend:Person {name: $friend_name}) "
#         "MERGE (a)-[:KNOWS]->(friend)",
#         name=name, friend_name=friend_name, database_="neo4j",
#     )


# def print_friends(driver, name):
#     records, _, _ = driver.execute_query(
#         "MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
#         "RETURN friend.name ORDER BY friend.name",
#         name=name, database_="neo4j", routing_=RoutingControl.READ,
#     )
#     for record in records:
#         print(record["friend.name"])


# with GraphDatabase.driver(URI, auth=AUTH) as driver:
#     add_friend(driver, "Arthur", "Guinevere")
#     add_friend(driver, "Arthur", "Lancelot")
#     add_friend(driver, "Arthur", "Merlin")
#     print_friends(driver, "Arthur")

In [7]:
driver = GraphDatabase.driver(URI, auth=AUTH)

In [8]:
# Get the Neo4j version
hneo4j.print_neo4j_version(driver)

Name: Neo4j Kernel, Version: ['5.22.0'], Edition: community


In [9]:
session = driver.session(database="neo4j")

In [10]:
# # TODO(gp): It seems that it's not easy to have multiple DBs in neo4j community edition.
# with driver.session(database="system") as session:
#     # Run the CREATE DATABASE command
#     session.run(f"CREATE DATABASE {database_name}")
#     print(f"Database '{database_name}' created successfully.")

# # Example usage
# database_name = "wine"

In [11]:
hneo4j.print_graph_stats(driver)

Number of nodes: 1
Number of edges: 0


# Example

- Every application using Neo4j needs a `driver` object
- A `driver` object holds the details to the connection to a Neo4j database (e.g., URIs, credentials, and configuration).

In [12]:
# Test the connection to the DB without executing any query.
driver.verify_connectivity()

In [13]:
driver.get_server_info()

### Populate the graph with one node.

In [14]:
hneo4j.print_graph_stats(driver)
print("Deleting ...")
hneo4j.delete_all(driver)
hneo4j.print_graph_stats(driver)

Number of nodes: 1
Number of edges: 0
Deleting ...
Number of nodes: 0
Number of edges: 0


In [15]:
query = 'CREATE (w:Wine {name:"Prancing Wolf", style: "ice wine", vintage: 2015})'
_ = driver.execute_query(query)
hneo4j.print_graph_stats(driver)

Number of nodes: 1
Number of edges: 0


### Performing a query using `driver`

In [16]:
# `execute_query()` wraps lower level APIs (e.g., Sessions) and it's used for simple cases.
query = "MATCH(n) RETURN COUNT(n) AS node_count"
result = driver.execute_query(query)

In [20]:
print(to_str(result))

records:
    1 [
            record=<int> <int> 1
    ]
keys:
    1 [
            <str> node_count
    ]



In [ ]:
# The returned object is of type `Result`.
print_(result, "result")

### 

In [ ]:
result[0][0]["node_count"]

In [ ]:
print(to_str(result[0]))

In [ ]:
print(to_str(result))

In [ ]:
# The result contains information about the query results and summary of the query.
records, summary, keys = result

# `result.records` is the list of records returned by the query.
print_(records, "records")
print_(summary, "summary")
# `result.keys` is the list of keys returned by the query.
print_(keys, "keys")

In [ ]:
# Extract the first `record` returned by the query.
print_(records[0])

In [ ]:
# Access the result.
records[0]["node_count"]

In [ ]:
# Return a node.
query = "MATCH(n:Wine) RETURN n"
result = driver.execute_query(query)
print_result(result)

In [ ]:
records = result[0]
print_(records, "records")
node = records[0]["n"]
print_(node, "node")

In [ ]:
#to_str(node)
print(to_str(records))
#dict(records[0]["n"].items())

In [ ]:
record = result[0][0]

In [ ]:
record.keys()

In [ ]:
# Access the properties of the node.
print(node["vintage"])
print(node["name"])
print(node["style"])

In [ ]:
# Return records.
query = "MATCH(n:Wine) RETURN n.name AS name, n.style as style"
result = driver.execute_query(query)
#
records = result[0]
print_(records[0], "records[0]")
#
record = records[0]
print_(record, "record")
#print_(record[0])

## Session

- Database activity is coordinated through `Session`s and `Transaction`s
- A `Session` is a container for a number of unit of works
    - Provide guarantees of causal consistency
    - Are lightweight opeation and not thread safe
- A `Transaction` is a unit of work that is either committed in its entirety or rolled back in case of failure

In [ ]:
# Create `Session`.
session = driver.session(database="neo4j")
print("session.closed()=", session.closed())

# Run a query.
query = "MATCH (n) RETURN n"
_ = session.run(query)

# Close `Session`.
session.close()
print("session.closed()=", session.closed())

In [ ]:
# Session can be created and destroyed using a block context, so that the session is closed
# properly in case of exceptions.
with driver.session() as session:
    result = session.run("MATCH (n) RETURN n")
    # ...

In [ ]:
# Create `Session`.
session = driver.session(database="neo4j")
print("session.closed()=", session.closed())

- `driver.execute_query()` is a higher-level function introduced to simplify query execution,
  without needing to manage sessions and transactions explicitly.
- `session.run()` is used for executing queries within a specific session and it
  provides more control over the session and transaction lifecycle.                                             

In [ ]:
# Count the number of nodes.
query = "MATCH(n) RETURN COUNT(n) AS node_count"

result = driver.execute_query(query)
print(type(result))
print(result)

- The returned result is typically a `neo4j.Result` object, which encapsulates the records, summary, and keys of the query execution.

In [ ]:
# Parse the result into its components.
records, summary, keys = result
print(type(records), records)
print(type(summary), summary)
print(type(keys), keys)

- `neo4j._data.Record` is a class in the Neo4j Python driver that represents a
  single row of results returned from a Cypher query
- Each `Record` object contains a series of named fields, corresponding to the
  columns of the result set

## Create 2 nodes

In [ ]:
hneo4j.print_graph_stats(driver)
print("Deleting ...")
hneo4j.delete_all(driver)
hneo4j.print_graph_stats(driver)

In [ ]:
# `w` has `Wine` label and then various properties.
query = 'CREATE (w:Wine {name:"Prancing Wolf", style: "ice wine", vintage: 2015})'
_ = driver.execute_query(query)

In [ ]:
# Create a node representing a publication.
query = 'CREATE (p:Publication {name: "Wine Expert Monthly"})'
_ = driver.execute_query(query)

In [ ]:
# Since the publication reports on the wine, we can create an edge.
query = '''
    MATCH (p:Publication {name: "Wine Expert Monthly"}),
      (w:Wine {name: "Prancing Wolf", vintage: 2015})
      CREATE (p)-[r:reported_on]->(w)
    '''
_ = driver.execute_query(query)

In [ ]:
hneo4j.print_graph_stats(driver)

In [ ]:
# Return a node.
#query = "MATCH(n:Wine) RETURN n"
#query = "MATCH (p:Publication) RETURN p"
query = "MATCH (p) RETURN p"
result = driver.execute_query(query)
#print_result(result)
records = result[0]
print(len(records), records)

In [ ]:
def to_str(obj):
    if isinstance(obj, Record):
        print

In [ ]:
# Match a relationship.
query = """
    MATCH ()-[r]-()
    RETURN r
    """
result = driver.execute_query(query)
#print_result(result)
records = result[0]
#print(len(records), records)
record = records[0]
print_(record[0], "record")

relationship = record[0]
assert str(type(relationship)) == "<class 'abc.reported_on'>"

print(relationship.element_id)
# I guess it doesn't want to print/retrieve too much info from the nodes, but only
# keeps the internal IDs.
print(relationship.start_node)
print(relationship.end_node)

In [ ]:
query = """
    MATCH (a)-[r]->(b)
    WHERE a.name = 'Wine Expert Monthly' AND b.name = 'Prancing Wolf'
    RETURN r;
"""
result = driver.execute_query(query)
#print_result(result)
records = result[0]
print(len(records), records)

In [ ]:
# The edge direction matter, in fact there is no edge "Prancing Wolf" -> "Wine Expert Monthly",
# but only the other direction.
query = """
    MATCH (a)-[r]->(b)
    WHERE a.name = 'Prancing Wolf' AND b.name = 'Wine Expert Monthly'
    RETURN r;
"""
result = driver.execute_query(query)
#print_result(result)
records = result[0]
print(len(records), records)

In [ ]:
# Search both direction.
query = """
    MATCH (a)-[r]-(b)
    WHERE a.name = 'Prancing Wolf' AND b.name = 'Wine Expert Monthly'
    RETURN r;
"""
result = driver.execute_query(query)
#print_result(result)
records = result[0]
print(len(records), records)